In [2]:
import torch.optim as optim
from MSD import MSDNet  # 假设 MSD 网络定义在 msd_pytorch_network.py 中
import torch.nn.functional as F

# 初始化 MSD 网络
msd_model = MSDNet(input_channels=1, output_channels=1, num_layers=5).cuda()  # 使用 GPU 加速

# 定义优化器和损失函数
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(msd_model.parameters(), lr=1e-4)

# 训练循环
num_epochs = 10  # 自定义训练轮数
msd_model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, inputs in enumerate(train_loader):
        inputs = inputs.cuda()  # Shape: [batch_size, channels, depth, height, width]
        
        # 初始化输出列表
        outputs_list = []
        
        # 按切片逐层处理
        for d in range(inputs.shape[2]):  # 遍历 depth 轴
            slice_2d = inputs[:, :, d, :, :]  # Shape: [batch_size, channels, height, width]
            output_2d = msd_model(slice_2d)
            outputs_list.append(output_2d.unsqueeze(2))  # 恢复 depth 维度
            
        # 拼接 3D 输出
        outputs = torch.cat(outputs_list, dim=2)  # Shape: [batch_size, channels, depth, height, width]
        
        # 上采样输出到 288x288x288
        outputs_resized = F.interpolate(outputs, size=(288, 288, 288), mode='trilinear', align_corners=False)
        
        # 目标张量
        target = F.interpolate(inputs, size=(288, 288, 288), mode='trilinear', align_corners=False).cuda()
        
        # 计算损失
        optimizer.zero_grad()
        loss = criterion(outputs_resized, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")


Epoch 1/10, Loss: 0.0660
Epoch 2/10, Loss: 0.0310
Epoch 3/10, Loss: 0.0107
Epoch 4/10, Loss: 0.0060
Epoch 5/10, Loss: 0.0077
Epoch 6/10, Loss: 0.0065
Epoch 7/10, Loss: 0.0041
Epoch 8/10, Loss: 0.0034
Epoch 9/10, Loss: 0.0035
Epoch 10/10, Loss: 0.0033


In [3]:
import nibabel as nib

msd_model.eval()
with torch.no_grad():
    for i, inputs in enumerate(test_loader):
        inputs = inputs.cuda()
        
        # 网络推理
        output = msd_model(inputs)
        
        # 将输出保存为 NIfTI 文件
        output_np = output.squeeze(0).cpu().numpy()  # 去掉 batch 和 channel 维度
        nii_image = nib.Nifti1Image(output_np, affine=np.eye(4))  # 创建 NIfTI 文件
        nib.save(nii_image, f"output_{i}.nii")
        print(f"Saved output_{i}.nii")


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 1, 288, 288, 36]